Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Input Directory
- This is the BIDS-style directory which was created by Notebook 02

In [6]:
# Shared Base Directory
base_directory = r'/Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds'


This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */*/tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [7]:
shared_glob_path = r'*/*/unthresholded_tissue_segment_z_scores'

This is the list of tissue types to use from within each terminal folder. 
- tissue_to_import = ['ct']
- **Leave as unchanged if you are unsure**

In [8]:

tissue_to_import = ['ct']

In [9]:
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
def import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import):
    """
    Imports dataframes based on tissue types from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - shared_glob_path (str): The shared directory path for all tissues.
    - tissue_to_import (list): List of tissues to be imported.
    
    Returns:
    - dict: A dictionary containing dataframes for each specified tissue.
    """

    segments_dict = {}
    for tissue in tissue_to_import:
        glob_path = os.path.join(shared_glob_path, ('*'+tissue+'*'))
        segments_dict[tissue] = glob_path

    dataframes_dict = {}
    nan_handler = {'nan': 0, 'posinf':20, 'neginf':-20}
    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=base_directory, file_pattern=v, convert_nan_to_num=nan_handler)
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'Example filename per subject: {dataframes_dict[k].columns[0]}')
        print('\n--------------------------------\n')

    return dataframes_dict


In [10]:
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/*/*/unthresholded_tissue_segment_z_scores/*ct*
Imported data ct data with 902629 voxels and 50 patients
Example filename per subject: sub-0001_ct.nii

--------------------------------



**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

In [11]:
from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id
def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """

    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


This Should Often Be Left Default

In [12]:
preceding_id = 'sub-'
proceeding_id = '_ct'

In [13]:
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  ct


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [16]:
folder_to_import_from = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois'
shared_glob_pattern = '*'

In [18]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois/*


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [19]:
import pandas as pd
import numpy as np
def calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df, count_voxels=False):
    """
    Calculates damage scores for each region of interest based on thresholded atrophy data.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - region_of_interest_df (DataFrame): DataFrame containing regions of interest.
    
    Returns:
    - dict: A dictionary containing damage scores for each region of interest for each patient.
    """
    
    damage_df_dict = {}
    
    for k, v in thresholded_atrophy_df_dict.items():
        
        # Initialize the Dataframe
        damage_df_dict[k] = pd.DataFrame(index=v.columns, columns=region_of_interest_df.columns)
        
        # Iterate Over Each Region of Interest
        for matrix in damage_df_dict[k].columns:
            # Extract Damage Score for Each Patient
            for subject in damage_df_dict[k].index:
                # Mask the subject dataframe to the matrix at hand
                intersection = v[subject].where(region_of_interest_df[matrix] > 0, 0)
                
                # Use multiplication to zero values outside ROI
                weighted_overlap = intersection * region_of_interest_df[matrix]
                
                # Assess overall damage score & atrophied voxels within ROI
                damage_score = weighted_overlap.sum()
                num_voxels = np.count_nonzero(weighted_overlap)
                
                # Assign values to DF 
                damage_df_dict[k].loc[subject, matrix] = damage_score
                damage_df_dict[k].loc[subject, 'num_atrophied_voxels_'+matrix] = num_voxels
                
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [20]:
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)


Dataframe:  ct


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,num_atrophied_voxels_mni_Cerebellum.nii.gz,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz
0001,0.0,-103.126604,-131.50633,-32.592013,-234.152299,-28.743475,-73.634952,18.485996,-164.171333,32.26009,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0002,0.0,-495.628398,-162.263637,-272.789836,-659.365101,-14.589434,-147.565307,73.194671,-234.214,-4.676519,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0003,0.0,-1584.16835,-395.42969,-399.128606,-1981.071107,-17.021116,-321.713134,-417.532386,-834.929279,33.921324,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0004,0.0,-1777.688965,-447.462585,-193.896495,-2226.624617,-16.344549,-290.050289,-876.163804,-810.30314,25.441995,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0005,0.0,-2490.204432,-570.927929,-513.829529,-3062.605428,-15.703671,-248.413262,-1270.632653,-954.511639,-31.589222,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0006,0.0,-2440.244837,-713.316715,-436.609648,-3140.261651,-7.963799,-368.081177,-1039.695716,-1215.666652,16.647405,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0007,0.0,-506.044992,-90.44394,-102.392715,-597.961999,-15.130805,-106.723719,-68.647318,-252.303643,-30.927755,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0008,0.0,106.143908,-60.54663,-235.940855,44.124211,-18.766152,104.043404,222.856722,80.06631,1.108423,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0009,0.0,-943.544175,-297.354451,-417.805122,-1240.279296,-17.667389,-75.677702,-124.484298,-550.921087,-25.293585,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0
0010,0.0,-1553.426927,-520.587004,-449.957156,-2075.486997,-14.450381,-159.654679,-580.65333,-775.268556,-16.255635,0.0,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0


------------- 



Extract Total Atrophy Volume

In [21]:
import numpy as np
from nimlab import datasets as nimds
def calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict):
    """
    Calculates the total number of atrophy voxels within a mask and updates the damage_df_dict with this information.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - damage_df_dict (dict): Dictionary containing dataframes to which the 'Total Atrophy Voxels' column will be added.
    
    Returns:
    - dict: The updated damage_df_dict with a new column 'Total Atrophy Voxels'.
    """
    
    # Get the mask and brain indices
    mni_mask = nimds.get_img("mni_icbm152")
    mask_data = mni_mask.get_fdata().flatten()
    brain_indices = np.where(mask_data > 0)[0]
    
    for k, v in thresholded_atrophy_df_dict.items():
        # initialize the column
        damage_df_dict[k]['Total Atrophy Voxels'] = np.nan
        for patient_id in v.columns:
            # Filter the dataframe using brain indices
            filtered_df = v[patient_id].iloc[brain_indices]
            damage_df_dict[k].loc[patient_id, 'Total Atrophy Voxels'] = np.count_nonzero(filtered_df)
        
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [22]:
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  ct


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
0001,0.0,-103.126604,-131.50633,-32.592013,-234.152299,-28.743475,-73.634952,18.485996,-164.171333,32.26009,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0002,0.0,-495.628398,-162.263637,-272.789836,-659.365101,-14.589434,-147.565307,73.194671,-234.214,-4.676519,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0003,0.0,-1584.16835,-395.42969,-399.128606,-1981.071107,-17.021116,-321.713134,-417.532386,-834.929279,33.921324,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0004,0.0,-1777.688965,-447.462585,-193.896495,-2226.624617,-16.344549,-290.050289,-876.163804,-810.30314,25.441995,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0005,0.0,-2490.204432,-570.927929,-513.829529,-3062.605428,-15.703671,-248.413262,-1270.632653,-954.511639,-31.589222,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0006,0.0,-2440.244837,-713.316715,-436.609648,-3140.261651,-7.963799,-368.081177,-1039.695716,-1215.666652,16.647405,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0007,0.0,-506.044992,-90.44394,-102.392715,-597.961999,-15.130805,-106.723719,-68.647318,-252.303643,-30.927755,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0008,0.0,106.143908,-60.54663,-235.940855,44.124211,-18.766152,104.043404,222.856722,80.06631,1.108423,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0009,0.0,-943.544175,-297.354451,-417.805122,-1240.279296,-17.667389,-75.677702,-124.484298,-550.921087,-25.293585,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
0010,0.0,-1553.426927,-520.587004,-449.957156,-2075.486997,-14.450381,-159.654679,-580.65333,-775.268556,-16.255635,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0


------------- 



Organize the Subjects

In [23]:
def sort_dataframes_by_index(damage_df_dict):
    """
    Attempts to convert the index of each dataframe in damage_df_dict to integers 
    and then sorts the dataframe by its index in ascending order.
    
    Parameters:
    - damage_df_dict (dict): Dictionary containing dataframes to be sorted.
    
    Returns:
    - dict: The sorted damage_df_dict.
    """
    
    sorted_df_dict = {}
    
    for k, df in damage_df_dict.items():
        try:
            # Convert index to integers
            df.index = df.index.astype(int)
            
            # Sort dataframe by index
            sorted_df = df.sort_index(ascending=True)
            sorted_df_dict[k] = sorted_df
            
            # Display the results
            print('Dataframe: ', k)
            display(sorted_df)
            print('------------- \n')
            
        except ValueError:
            # If conversion to integer fails, just use the original dataframe
            sorted_df_dict[k] = df

    return sorted_df_dict


In [24]:
sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)

Dataframe:  ct


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
1,0.0,-103.126604,-131.50633,-32.592013,-234.152299,-28.743475,-73.634952,18.485996,-164.171333,32.26009,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
2,0.0,-495.628398,-162.263637,-272.789836,-659.365101,-14.589434,-147.565307,73.194671,-234.214,-4.676519,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
3,0.0,-1584.16835,-395.42969,-399.128606,-1981.071107,-17.021116,-321.713134,-417.532386,-834.929279,33.921324,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
4,0.0,-1777.688965,-447.462585,-193.896495,-2226.624617,-16.344549,-290.050289,-876.163804,-810.30314,25.441995,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
5,0.0,-2490.204432,-570.927929,-513.829529,-3062.605428,-15.703671,-248.413262,-1270.632653,-954.511639,-31.589222,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
6,0.0,-2440.244837,-713.316715,-436.609648,-3140.261651,-7.963799,-368.081177,-1039.695716,-1215.666652,16.647405,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
7,0.0,-506.044992,-90.44394,-102.392715,-597.961999,-15.130805,-106.723719,-68.647318,-252.303643,-30.927755,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
8,0.0,106.143908,-60.54663,-235.940855,44.124211,-18.766152,104.043404,222.856722,80.06631,1.108423,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
9,0.0,-943.544175,-297.354451,-417.805122,-1240.279296,-17.667389,-75.677702,-124.484298,-550.921087,-25.293585,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0
10,0.0,-1553.426927,-520.587004,-449.957156,-2075.486997,-14.450381,-159.654679,-580.65333,-775.268556,-16.255635,...,1799.0,523.0,393.0,2327.0,382.0,269.0,777.0,697.0,43.0,2325.0


------------- 



Save the Results

In [25]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from tqdm import tqdm

def save_csv_to_bids(dataframes_dict, bids_base_dir, analysis='atrophy_results', ses=None, dry_run=True):
    """
    Saves csv to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the CSV 
    accordingly.
    """
    
    for key, value in tqdm(dataframes_dict.items()):            
        # Define BIDS Directory Architecture
        sub_no = 'all'
        if ses is None:
            ses_no = '01'
        else:
            ses_no = ses
        
        # Define and Initialize the Save Directory
        out_dir = os.path.join(bids_base_dir, 'neuroimaging_analyses', f'ses-{ses_no}', f'sub-{sub_no}', analysis)
        os.makedirs(out_dir, exist_ok=True)
        
        # Save Image to BIDS Directory
        if dry_run:
            print(out_dir+f'/{key}.csv')
        else:
            value.to_csv(out_dir+f'/{key}.csv')
            print('Saved to: ', out_dir+f'/{key}.csv')

In [26]:
analysis='unthresholded_ct_atrophy_results'

In [28]:
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir=base_directory, analysis=analysis, ses=None, dry_run=False)

100%|██████████| 1/1 [00:00<00:00,  7.98it/s]

Saved to:  /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/neuroimaging_analyses/ses-01/sub-all/thresholded_atrophy_results/ct.csv


All done. Enjoy your analyses. 

--Calvin 